In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from io import StringIO
from findCorrespondance import findCorrespondance
from GetInliersRANSAC import *
from EssentialMatrixFromFundamentalMatrix import *

#### Reject Outliers and Estimate Fundamental Matrix

In [2]:
elimination_threshold = 5
all_inliers = []
for i in range(1, 3):
    for j in range(i, 3):
        if (i != j):
            print("\n ---------- \n\n Finding Correspondance between image " + str(i)
                + " and " + str(j) + ":")
            matching_list = findCorrespondance(i, j)
            correspondence_list = [[row[3]for row in matching_list],
            [row[4]for row in matching_list],
            [row[5]for row in matching_list], [row[6]for row in matching_list]]
#             correspondence_list = np.transpose(correspondence_list)
            # print("shape(correspondence_list): ",
                # np.shape(correspondence_list))
            finalF, inliers = GetInliersRANSAC(correspondence_list, elimination_threshold)
            print("Final F = ", finalF)


 ---------- 

 Finding Correspondance between image 1 and 2:
Into GetInliersRANSAC now,
Shape of corr_list = 1426
Running Ransac for 0/1000
Running Ransac for 100/1000


KeyboardInterrupt: 

In [ ]:
inliers

In [ ]:
            img1 = cv2.imread('Data/' + str(1) + '.jpg')
            img2 = cv2.imread('Data/' + str(2) + '.jpg')

            rows1, cols1 = img1.shape[:2]
            rows2, cols2 = img2.shape[:2]
            out = np.zeros((max([rows1, rows2]), cols1+cols2, 3), dtype='uint8')
            out[:rows1, :cols1, :] = img1
            out[:rows2, cols1:cols1+cols2, :] = img2

In [ ]:
cv2.imshow("out", out)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
            kp1 = [[row[0] for row in inliers], [row[1] for row in inliers]]
            kp1 = np.transpose(kp1)
            kp2 = [[row[2] for row in inliers], [row[3] for row in inliers]]
            kp2 = np.transpose(kp2)
            print("Shape kp1 = ", np.shape(kp1))

In [ ]:
len(kp1)

In [ ]:
            
            radius = 4
            BLUE = (255, 0, 0)
            GREEN = (0,255,0)
            thickness = 2
            for i in range(0, len(kp1)):
                # Draw small circle on image 1
                cv2.circle(out, (int(kp1[i][0]),int(kp1[i][1])), radius, BLUE, thickness)
                
                # Draw small circle on image 2
                cv2.circle(out, (int(kp2[i][0])+cols1,int(kp2[i][1])), radius, GREEN, thickness)
                
                # Draw line connecting circles
                cv2.line(out,(int(kp1[i][0]),int(kp1[i][1])),(int(kp2[i][0])+cols1,int(kp2[i][1])),BLUE,thickness)
                
                
            # draw_params = dict(matchColor=(0, 255, 0),  # draw matches in green color
            #        singlePointColor=None,
            #        matchesMask=out,  # draw only inliers
            #        flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT)


            # img3 = cv2.drawMatches(img1, kp1, img2, kp2, [match[0] for match in inliers], None, draw_params)
            # , good, None, **draw_params)
            cv2.imshow("img3", out)

            cv2.waitKey(300)
            cv2.destroyAllWindows()
# A = np.array(correspondence_list)
# print(A.shape)